In [1]:
from pvpf.model.convlstm import build_conv_lstm
from pvpf.tfrecord.high_level import load_dataset
from pvpf.token.training_token import TRAINING_TOKENS
import tensorflow as tf

batch_size = 4
num_epoch = 3
training_property = TRAINING_TOKENS["base"]
train_x, test_x, train_y, test_y = load_dataset(training_property)
train_dataset = tf.data.Dataset.zip((train_x, train_y))
test_dataset = tf.data.Dataset.zip((test_x, test_y))
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
model = build_conv_lstm()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss="mse")
model.fit(train_dataset, validation_data=test_dataset, epochs=num_epoch)

2021-09-10 14:01:41.068697: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-10 14:01:43.084913: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-09-10 14:01:43.132439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 14:01:43.134180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 SUPER computeCapability: 7.5
coreClock: 1.65GHz coreCount: 34 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2021-09-10 14:01:43.134291: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-09-10 14:01:43.155861: I tensorflow/stream_execu

Epoch 1/3


2021-09-10 14:01:47.321517: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-09-10 14:01:47.979531: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-09-10 14:01:49.536048: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-09-10 14:01:50.343661: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


1649/1649 [==============================] - 980s 590ms/step - loss: 44368.6406 - val_loss: 43689.5938
Epoch 2/3
 290/1649 [====>.........................] - ETA: 11:42 - loss: 83548.9688

KeyboardInterrupt: 

In [ ]:
from typing import Tuple
import numpy as np
def get_prediction(model: tf.keras.models.Model, dataset: tf.data.Dataset) -> Tuple[np.ndarray, np.ndarray]:
    ys = list()
    ts = list()
    count = 0
    for x, t in dataset:
        y = model(x)
        ys.append(y.numpy())
        ts.append(t.numpy())
        count += len(t.numpy())
    ys = np.concatenate(ys, 0)
    ts = np.concatenate(ts, 0)
    return ys, ts

In [ ]:
from pvpf.utils.date_range import date_range
import pandas as pd
from pathlib import Path
predictions, targets = get_prediction(model, test_dataset)
datetimes = list(date_range(training_property.prediction_split, training_property.prediction_end, training_property.tfrecord_property.time_unit))
df_dict = {"datetime": datetimes, "prediction": predictions, "target": targets}
df = pd.DataFrame(df_dict)
path = Path(".").joinpath("output", "sep7.csv")
df.to_csv(path)

In [7]:
count = 0
for x, y in test_dataset:
    count += 1
    if count % 100 == 0:
        print(x.numpy()[0][0][0][0])

[354.352       19.5312      -0.70710677   0.70710677   0.
   0.8660254    0.5         70.7761      19.5312     284.09      ]
[0.0000000e+00 0.0000000e+00 9.6592581e-01 2.5881904e-01 1.7138700e+01
 5.0000000e-01 8.6602539e-01 7.0479103e+01 1.7138700e+01 2.8360199e+02]
[ 3.7500000e+01  0.0000000e+00 -2.5881904e-01 -9.6592581e-01
  0.0000000e+00  5.0000000e-01  8.6602539e-01  6.6660301e+01
  0.0000000e+00  2.8350900e+02]
[ 1.1076000e+02  3.5156200e+00 -7.0710677e-01  7.0710677e-01
  9.0185501e+01  6.1232343e-17  1.0000000e+00  7.7379204e+01
  9.0185501e+01  2.8744000e+02]
[0.0000000e+00 9.8095703e+01 9.6592581e-01 2.5881904e-01 3.8281200e+01
 6.1232343e-17 1.0000000e+00 6.6812302e+01 9.8095703e+01 2.8879800e+02]
